In [23]:
import os
import glob
from sklearn.model_selection import train_test_split

In [24]:
from scipy.spatial.qhull import QhullError
from scipy import spatial
spatial.QhullError = QhullError

/tmp/ipykernel_33/2927579722.py:1: DeprecationWarning: Please import `QhullError` from the `scipy.spatial` namespace; the `scipy.spatial.qhull` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.spatial.qhull import QhullError


In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from category_encoders import OrdinalEncoder

In [26]:
import os
import glob

def get_image_paths(folder_path):
    image_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image_paths.append(image_path)
    return image_paths

if __name__ == "__main__":
    # Specify input folders
    input_folder_blacksigatoka = r"/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka"
    input_folder_furasiumwilt = r"/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt"
    input_folder_cordana = r"/kaggle/input/blddisease-wise/BLD_augmented_data/Cordana"

    # Get image paths for each class
    image_paths_blacksigatoka = get_image_paths(input_folder_blacksigatoka)
    image_paths_furasiumwilt = get_image_paths(input_folder_furasiumwilt)
    image_paths_cordana = get_image_paths(input_folder_cordana)

    # Print the first few image paths for each class
    print("Image paths for BlackSigatoka:")
    print(image_paths_blacksigatoka[:5])

    print("\nImage paths for FurasiumWilt:")
    print(image_paths_furasiumwilt[:5])

    print("\nImage paths for Cordana:")
    print(image_paths_cordana[:5])


Image paths for BlackSigatoka:
['/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/257_jpeg.rf.3056e638b36d2805190c731d35590a75_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/267_jpeg.rf.b449a72add174c60aaef9595acd9e1f0_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/Image_1650_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/Image_1672_aug_1.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka/202_jpeg.rf.4cae05ab9f7463e25bc30a6a4fdd0883_aug_0.png']

Image paths for FurasiumWilt:
['/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_3243_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_106_aug_1.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_2922_aug_0.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt/Image_3277_aug_1.png', '/kaggle/input/blddisease-wise/BLD_augmented_data/Furasi

In [27]:
import os
import cv2
import numpy as np
from skimage import exposure
from tqdm import tqdm

def preprocess_images(file_paths, output_folder, target_size=(64, 64)):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for path in tqdm(file_paths):
        # Read the image
        image = cv2.imread(path)

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Histogram Equalization to enhance contrast
        equalized = exposure.equalize_hist(gray)

        # Resize the image to a standard size
        resized = cv2.resize(equalized, target_size)

        # Normalize pixel values to be between 0 and 1
        resized = resized / 255.0

        # Save the preprocessed image
        filename = os.path.basename(path)
        output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_preprocessed.png")
        cv2.imwrite(output_path, (resized * 255).astype(np.uint8))

if __name__ == "__main__":
    # Specify input and output folders
    input_folder_blacksigatoka = r"/kaggle/input/blddisease-wise/BLD_augmented_data/BlackSigatoka"
    input_folder_furasiumwilt = r"/kaggle/input/blddisease-wise/BLD_augmented_data/FurasiumWilt"
    input_folder_cordana = r"/kaggle/input/blddisease-wise/BLD_augmented_data/Cordana"
    output_folder_cordana = r"/kaggle/working/BLD_augmented_data2/Cordana"
    output_folder_blacksigatoka = r"/kaggle/working/BLD_augmented_data2/BlackSigatoka"
    output_folder_furasiumwilt = r"/kaggle/working/BLD_augmented_data2/FurasiumWilt"

    # Get image paths for each class
    image_paths_blacksigatoka = [os.path.join(input_folder_blacksigatoka, filename) for filename in os.listdir(input_folder_blacksigatoka)]
    image_paths_furasiumwilt = [os.path.join(input_folder_furasiumwilt, filename) for filename in os.listdir(input_folder_furasiumwilt)]
    image_paths_cordana = [os.path.join(input_folder_cordana, filename) for filename in os.listdir(input_folder_cordana)]

    # Preprocess images
    preprocess_images(image_paths_blacksigatoka, output_folder_blacksigatoka)
    preprocess_images(image_paths_furasiumwilt, output_folder_furasiumwilt)
    preprocess_images(image_paths_cordana, output_folder_cordana)


100%|██████████| 324/324 [00:01<00:00, 168.51it/s]


In [28]:
import os
import glob
from skimage import io, feature
import numpy as np
from skimage import io, feature, transform
import numpy as np
from skimage.color import rgb2gray
from scipy.fft import fft2, fftshift

X_train = []  # Initialize X_train
y_train = []  # Initialize y_train

def extract_hog_lbp_features(image_path,spatial_binning_size=(32,32)):
    image = io.imread(image_path, as_gray=True)
    image = (image * 255).astype(np.uint8)
    # Resize the image for spatial binning
    resized_image = transform.resize(image, spatial_binning_size, anti_aliasing=True)
    # gradient binning 
    hog_features = feature.hog(resized_image, block_norm='L2-Hys', visualize=False, pixels_per_cell=(8, 8))
    lbp_features = feature.local_binary_pattern(image, P=16, R=3, method='uniform')
    return np.concatenate((hog_features, lbp_features.flatten()))

def extract_fourier_features(image_path):
    image = io.imread(image_path, as_gray=True)  # Read the image as grayscale
    fft_image = fft2(image)
    fft_shifted = fftshift(fft_image)
    magnitude_spectrum = np.log(np.abs(fft_shifted) + 1)  # Log-scaled magnitude spectrum
    return magnitude_spectrum.flatten()


def extract_features(input_folder):
    file_paths = glob.glob(os.path.join(input_folder, '*.png'))
    for image_path in file_paths:
        hog_lbp_features = extract_hog_lbp_features(image_path)
        fourier_features = extract_fourier_features(image_path)
        all_features = np.concatenate((hog_lbp_features, fourier_features))
        X_train.append(all_features)
        
        # Assuming the label is the name of the class folder
        label = os.path.basename(os.path.dirname(image_path))
        y_train.append(label)

if __name__ == "__main__":
    # Specify input and output folders
    input_folder_blacksigatoka = r"/kaggle/working/BLD_augmented_data2/BlackSigatoka"
    input_folder_furasiumwilt = r"/kaggle/working/BLD_augmented_data2/FurasiumWilt"
    input_folder_cordana = r"/kaggle/working/BLD_augmented_data2/Cordana"
    
    label_mapping = {'BlackSigatoka': 0, 'FurasiumWilt': 1, 'Cordana': 2}
    
#     X_blacksigatoka, y_blacksigatoka = extract_features(input_folder_blacksigatoka)
#     X_furasiumwilt, y_furasiumwilt = extract_features(input_folder_furasiumwilt)
#     X_cordana, y_cordana = extract_features(input_folder_cordana)

    #Extract HOG and LBP features
    extract_features(input_folder_blacksigatoka)
    extract_features(input_folder_furasiumwilt)
    extract_features(input_folder_cordana)

#     # Concatenate features and labels
#     X = np.concatenate((X_blacksigatoka, X_furasiumwilt, X_cordana), axis=0)
#     y = np.concatenate((y_blacksigatoka, y_furasiumwilt, y_cordana), axis=0)

    y_train_numeric = [label_mapping[label] for label in y_train]

    # Convert lists to arrays
    X_train_array = np.array(X_train)
    y_train_array = np.array(y_train_numeric)

    # Concatenate features and labels
    #all_features = np.column_stack((X_train_array, y_train_array))

    # Flatten the features
    #flattened_features = all_features.flatten()

    # Print the shapes of the extracted features
    print("Shape of HOG and LBP features (All Classes):", X_train_array.shape)
    print("Shape of labels (All Classes):", y_train_array.shape)
    #print("Shape of concatenated features and labels:", all_features.shape)
    #print("Shape of flattened features (All Classes):", flattened_features.shape)


Shape of HOG and LBP features (All Classes): (2414, 8516)
Shape of labels (All Classes): (2414,)


In [29]:
 y_train_array

array([0, 0, 0, ..., 2, 2, 2])

In [30]:
import numpy as np
from skimage import feature, io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers

In [31]:
# Apply z-score normalization
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train_array)
all_features = np.column_stack((X_train_normalized, y_train_array))

# Print the shape and first few values of the normalized features
print("Shape of normalized features (training set):", X_train_normalized.shape)
print("Shape of all features:", all_features.shape)
print("First few values of all features:", all_features[:])
print("First few values of normalized features (training set):", X_train_normalized[:])

Shape of normalized features (training set): (2414, 8516)
Shape of all features: (2414, 8517)
First few values of all features: [[-0.26001724 -0.15645309 -0.18569431 ... -0.56902984 -0.56665216
   0.        ]
 [-0.26001724 -0.15645309 -0.18569431 ... -0.56902984 -0.56665216
   0.        ]
 [-0.26001724 -0.15645309 -0.18569431 ...  1.4599342   1.49677842
   0.        ]
 ...
 [-0.2434231  -0.09533242 -0.1201166  ...  1.373465    1.52725917
   2.        ]
 [-0.26001724 -0.15645309 -0.18569431 ... -0.56902984 -0.56665216
   2.        ]
 [-0.26001724 -0.15645309 -0.18569431 ... -0.56902984 -0.56665216
   2.        ]]
First few values of normalized features (training set): [[-0.26001724 -0.15645309 -0.18569431 ... -0.57311893 -0.56902984
  -0.56665216]
 [-0.26001724 -0.15645309 -0.18569431 ... -0.57311893 -0.56902984
  -0.56665216]
 [-0.26001724 -0.15645309 -0.18569431 ...  1.49924499  1.4599342
   1.49677842]
 ...
 [-0.2434231  -0.09533242 -0.1201166  ...  2.57028435  1.373465
   1.52725917

In [32]:
import glob
from sklearn.model_selection import train_test_split

# Now, let's split the data
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(all_features, y_train_array, test_size=0.2, random_state=42)

# Print the sizes of the training and testing sets
print(f"Training set size: {len(X_train_split)}")
print(f"Testing set size: {len(X_test_split)}")

Training set size: 1931
Testing set size: 483


In [36]:
y_train_split

array([0, 2, 0, ..., 1, 1, 0])

In [37]:
X_train_split.shape[1]

8517

In [34]:
# import numpy as np
# from scipy.optimize import minimize

# # Define the Fitness Function
# def fitness_function(hyperparameters):
#     optimizer, loss, activation, metric = hyperparameters
    
#     # Build a neural network model
#     model = Sequential()
#     model.add(Dense(128, input_dim=X_train_split.shape[1], activation=activation))
#     model.add(Dense(64, activation=activation))
#     model.add(Dense(1, activation='sigmoid'))
    
#     # Compile the model
#     model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
    
#     # Train the model
#     model.fit(X_train_split, y_train_split, epochs=50, batch_size=32, validation_data=(X_test_split, y_test_split), verbose=0)
    
#     # Evaluate the model on the validation set
#     y_val_pred = (model.predict(X_test_split) > 0.5).astype(int)
#     val_accuracy = accuracy_score(y_test_split, y_val_pred)
    
#     return -val_accuracy  # PSO minimizes, so we use negative accuracy as the objective

# # Define the Bounds for PSO
# bounds = [(0, 1),  # optimizer (assuming it's a continuous variable between 0 and 1)
#           (0, 1),  # loss (assuming it's a continuous variable between 0 and 1)
#           (None, None),  # activation (assuming it can take any value)
#           (None, None)]  # metric (assuming it can take any value)

# # Run PSO Optimization
# result = minimize(fitness_function, x0=np.random.rand(4), bounds=bounds, method='L-BFGS-B')

# # Get the Best Hyperparameters
# best_params = result.x

# # Print the Best Hyperparameters
# print("Best Hyperparameters:", best_params)

# # Build and Train the Final Model
# final_model = Sequential()
# final_model.add(Dense(128, input_dim=X_train_split.shape[1], activation=best_params[2]))
# final_model.add(Dense(64, activation=best_params[2]))
# final_model.add(Dense(1, activation='sigmoid'))

# final_model.compile(loss=best_params[1], optimizer=best_params[0], metrics=[best_params[3]])
# final_model.fit(X_train_split, y_train_split, epochs=50, batch_size=32, validation_data=(X_test_split, y_test_split))


In [47]:
# import os
from skimage import io, feature
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from itertools import product  # Add this line
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt# Add this line for plotting histograms
    
    
# Define the hyperparameter combinations to try
optimizers = ['adam', 'sgd']
loss_functions = ['binary_crossentropy', 'mean_squared_error']
activation_functions = ['relu', 'sigmoid',]
metrics = ['accuracy']

X_train_split=np.array(X_train_split)
y_train_split=np.array(y_train_split)

best_accuracy = 0
best_params = {}

    # Perform grid search
for optimizer, loss, activation, metric in product(optimizers, loss_functions, activation_functions, metrics):
    # Build a simple neural network model
    model = Sequential()
    model.add(Dense(8517, input_dim=X_train_split.shape[1], activation=activation))
    model.add(Dense(1024, activation=activation))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    model.add(Dense(512, activation=activation))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    model.add(Dense(256, activation=activation))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    model.add(Dense(64, activation=activation))
#     model.add(Dropout(0.3))  # Adding dropout for regularization
    model.add(Dense(32, activation=activation))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    model.add(Dense(16, activation=activation))  # Changed activation to 'softmax' for 3 classes
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
    
    

    # Convert labels to one-hot encoding
#     y_train_split_encoded = to_categorical(y_train_split) 
#     y_test_split_encoded = to_categorical(y_test_split)

    # Train the model
    model.fit(X_train_split, y_train_split, epochs=100, batch_size=32, validation_data=(X_test_split, y_test_split))

    # Evaluate the model on the validation set
    y_val_pred = (model.predict(X_test_split) > 0.5).astype(int)
    val_accuracy = accuracy_score(y_test_split, y_val_pred)

    print(f"Optimizer: {optimizer}, Loss: {loss}, Activation: {activation}, Metric: {metric}, Validation Accuracy: {val_accuracy * 100:.2f}%")

    # Check if this combination resulted in a better accuracy
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_params = {'optimizer': optimizer, 'loss': loss, 'activation': activation, 'metric': metric}

    # Print the best hyperparameters
    print("\nBest Hyperparameters:")
    print(best_params)

    # Build the final model with the best hyperparameters
    final_model = Sequential()
    final_model.add(Dense(8517, input_dim=X_train_split.shape[1], activation=best_params['activation']))
    final_model.add(Dense(1024, activation=best_params['activation']))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    final_model.add(Dense(512, activation=best_params['activation']))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    final_model.add(Dense(256, activation=best_params['activation']))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    final_model.add(Dense(64, activation=best_params['activation']))
#     model.add(Dropout(0.3))  # Adding dropout for regularization
    final_model.add(Dense(32, activation=best_params['activation']))
#     final_model.add(Dropout(0.3))  # Adding dropout for regularization
    final_model.add(Dense(16, activation=best_params['activation']))  # Changed activation to 'softmax' for 3 classes
    final_model.add(Dense(1, activation='sigmoid'))

    # Compile the final model
    final_model.compile(loss=best_params['loss'], optimizer=best_params['optimizer'], metrics=[best_params['metric']])

    # Train the final model on the entire training set
    final_model.fit(X_train_split, y_train_split, epochs=100, batch_size=32)

    # Make predictions on the validation set
    y_val_pred = (final_model.predict(X_test_split) > 0.5).astype(int)

    # Evaluate the final model on the validation set
    accuracy = accuracy_score(y_test_split, y_val_pred)
    print(f"\nFinal Model Accuracy: {accuracy * 100:.2f}%")

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_test_split, y_val_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Classification Report
    class_report = classification_report(y_test_split, y_val_pred)
    print("Classification Report:")
    print(class_report)

# Plot histograms for the first 3 features
for i in range(3):  # Adjust this range based on the number of features you have
    plt.hist(X_train_split[y_train_split == 0][:, i], alpha=0.5, label='class1', bins=20, color='blue')
    plt.hist(X_train_split[y_train_split == 1][:, i], alpha=0.5, label='class2', bins=20, color='orange')
    plt.hist(X_train_split[y_train_split == 2][:, i], alpha=0.5, label='class3', bins=20, color='green')
    plt.title(f'Histogram for Feature {i + 1}')
    plt.xlabel(f'Feature {i + 1} Value')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()


Epoch 1/100
 7/61 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.4447 - loss: 5.7898

W0000 00:00:1710077774.494794     258 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.4545 - loss: 3.7546

W0000 00:00:1710077779.897217     255 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 95ms/step - accuracy: 0.4543 - loss: 3.7341 - val_accuracy: 0.4720 - val_loss: 1.2761
Epoch 2/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4591 - loss: 0.9093 - val_accuracy: 0.4327 - val_loss: 0.7284
Epoch 3/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.4625 - loss: -1.6084 - val_accuracy: 0.4534 - val_loss: 1.3943
Epoch 4/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4635 - loss: 1.1682 - val_accuracy: 0.4741 - val_loss: 0.6084
Epoch 5/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4570 - loss: -0.8900 - val_accuracy: 0.4286 - val_loss: 618.5636
Epoch 6/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4305 - loss: -22715.4922 - val_accuracy: 0.4720 - val_loss: -143696.8438
Epoch 7/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4366 - loss: -14130520.0000 - val_accuracy: 0.4720 - val_loss: -49859056.0000
Epoch 8/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4494 - loss: -16132

...

W0000 00:00:1710078760.349135     257 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.4537 - loss: 0.4737

W0000 00:00:1710078765.908480     257 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.4535 - loss: 0.4738 - val_accuracy: 0.4720 - val_loss: 0.4397
Epoch 2/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4383 - loss: 0.4863 - val_accuracy: 0.4720 - val_loss: 0.4388
Epoch 3/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4428 - loss: 0.4744 - val_accuracy: 0.4720 - val_loss: 0.4395
Epoch 4/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4272 - loss: 0.4946 - val_accuracy: 0.4720 - val_loss: 0.4384
Epoch 5/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4313 - loss: 0.4739 - val_accuracy: 0.4720 - val_loss: 0.4402
Epoch 6/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4525 - loss: 0.4692 - val_accuracy: 0.4720 - val_loss: 0.4398
Epoch 7/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4456 - loss: 0.4807 - val_accuracy: 0.4720 - val_loss: 0.4387
Epoch 8/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4529 - loss: 0.4646 - val_accuracy: 0.4720 - val_l

KeyboardInterrupt: 